In [1]:
from tamalero.FIFO import FIFO
from tamalero.ETROC import ETROC
from tamalero.LPGBT import LPGBT
from tamalero.utils import get_kcu
from tamalero.DataFrame import DataFrame
from tamalero.colors import green, red, yellow
from tamalero.ReadoutBoard import ReadoutBoard
import os
import sys
import tty
import time
import select
import pickle
import termios
import numpy as np
from tqdm import tqdm
from random import randint
from datetime import datetime, timezone

KCU_IP = "192.168.0.10" ## If your KCU ip is diff, modify it.

READOUTBOARD_ID = 0
READOUTBOARD_CONFIG = 'default'

ETROC_I2C_ADDRESSES = [0x63, 0x62, 0x61, 0x60]
ETROC_I2C_CHANNEL = 1
ETROC_ELINKS_MAP = {0: [0, 4, 8, 12]}


print('ETROC COSMIC RUN TEST - HARDWARE INITIALIZAATION')


kcu = get_kcu(
    KCU_IP,
    control_hub=True,
    host='localhost',
    verbose=False
)
print(green("Successfully connected to KCU."))

kcu.status() # Prints LpGBT link statuses from KCU 
# fw_ver = kcu.get_firmware_version() #
# kcu.check_clock_frequencies() # Verifies KCU clock stability

# Perform a simple loopback register test to confirm communication
loopback_val = 0xABCD1234
kcu.write_node("LOOPBACK.LOOPBACK", loopback_val) #
read_val = kcu.read_node("LOOPBACK.LOOPBACK").value()

if read_val == loopback_val:
    print(green(f"KCU Loopback test PASSED: Wrote 0x{loopback_val:X}, Read 0x{read_val:X}"))
else:
    print(red(f"KCU Loopback test FAILED: Wrote 0x{loopback_val:X}, Read 0x{read_val:X}"))

# ======================================================================================
# 2. INITIALIZE READOUT BOARD
# ======================================================================================
rb = ReadoutBoard(
    rb=READOUTBOARD_ID,
    kcu=kcu,
    config=READOUTBOARD_CONFIG, 
    trigger=False,     
    verbose=False,
    allow_bad_links=True
)
print(green(f"Readout Board version detected: {rb.ver}"))

# ======================================================================================
# 3. INITIALIZE FOUR ETROC
# ======================================================================================

print("\n3. Initializing ETROC chips...")
etroc_chips = []
chip_names = []

for i, addr in enumerate(ETROC_I2C_ADDRESSES):
    chip_name = f"Chip{i+1}"
    chip_names.append(chip_name)
    
    print(f"\nInitializing {chip_name} (I2C: 0x{addr:02X})...")
    
    try:
        etroc = ETROC(
            rb,
            master='lpgbt',
            i2c_adr=addr,
            i2c_channel=ETROC_I2C_CHANNEL,
            elinks=ETROC_ELINKS_MAP,
            strict=False,
            verbose=True
        )
        etroc_chips.append(etroc)
        
        # Verify communication
        if etroc.is_connected():
            # Check key registers
            scrambler_status = etroc.rd_reg("disScrambler")
            controller_state = etroc.rd_reg("controllerState")
            pll_unlock_count = etroc.rd_reg("pllUnlockCount")
            
            print(green(f"✓ {chip_name} connected successfully"))
            print(f"  Controller state: {controller_state} (should be 11)")
            print(f"  PLL unlock count: {pll_unlock_count}")
            
            if scrambler_status == 1:
                print(green("  ✓ Register communication verified"))
            else:
                print(red("  ✗ Register communication issue"))
        else:
            print(red(f"✗ {chip_name} not responding"))
            
    except Exception as e:
        print(red(f"✗ Failed to initialize {chip_name}: {e}"))
        etroc_chips.append(None)

print(green("\n✓ Hardware initialization completed successfully!"))
print(f"Initialized {len([c for c in etroc_chips if c is not None])} ETROC chips")


   

ETROC COSMIC RUN TEST - HARDWARE INITIALIZAATION
IPBus address: chtcp-2.0://localhost:10203?target=192.168.0.10:50001
KCU firmware version: 3.2.4
Successfully connected to KCU.
LPGBT Link Status from KCU:
0x2021  r       READOUT_BOARD_0.LPGBT.DOWNLINK.READY              0x00000001
0x2001  r       READOUT_BOARD_0.LPGBT.UPLINK_0.READY              0x00000000
0x2001  r       READOUT_BOARD_0.LPGBT.UPLINK_0.FEC_ERR_CNT        0x0000FFFF
KCU Loopback test PASSED: Wrote 0xABCD1234, Read 0xABCD1234
0x1d7 readback value is 0x00
lpGBT version found False
1
0x1d7 readback value is 0x00
lpGBT version found False
2
0x1d7 readback value is 0x00
lpGBT version found False
3
0x1d7 readback value is 0x00
lpGBT version found False
4
0x1d7 readback value is 0x00
lpGBT version found False
5
0x1d7 readback value is 0x00
lpGBT version found False
6
0x1d7 readback value is 0x00
lpGBT version found False
7
0x1d7 readback value is 0x00
lpGBT version found False
8
0x1d7 readback value is 0x00
lpGBT version found

Exception: Could not successfully read from lpGBT and failed to determine lpGBT version. Check optical links and power of RB.

In [2]:
print("\nETROC COSMIC RUN TEST - QINJ ONLY AND DATA ACQUISITION")
CHARGE_FC = 30 
QINJ_COUNT = 1000
MANUAL_OFFSET = 50

TRIGGER_ENABLE_MASK = 0x8
TRIGGER_DATA_SIZE = 1
TRIGGER_DELAY_SEL = 472

print("\n1. Generating test pixel configuration...")

# Generate random test pixels for each chip
test_pixels_per_chip = [
    [(randint(0, 15), randint(0, 15))],  # Chip1: 1 pixel
    [(randint(0, 15), randint(0, 15)), (randint(0, 15), randint(0, 15))],  # Chip2: 2 pixels
    [(randint(0, 15), randint(0, 15)), (randint(0, 15), randint(0, 15))],  # Chip3: 2 pixels
    [(randint(0, 15), randint(0, 15))]   # Chip4: 1 pixel
]

etroc_configs = []
for i, (etroc, chip_name) in enumerate(zip(etroc_chips, chip_names)):
    if etroc is not None and i < len(test_pixels_per_chip):
        etroc_configs.append((etroc, chip_name, test_pixels_per_chip[i]))

print("Test pixel assignments:")
for etroc, chip_name, pixels in etroc_configs:
    print(f"  {chip_name}: {pixels}")


print("\n2. Calibrating pixel baselines...")
baseline_storage = {}

for etroc, chip_name, test_pixels in etroc_configs:
    baseline_storage[chip_name] = {}
    print(f"\nScanning {chip_name}...")
    
    for pixel_row, pixel_col in test_pixels:
        print(f"  Calibrating pixel ({pixel_row}, {pixel_col})...")
        
        baseline, noise_width = etroc.auto_threshold_scan(
            row=pixel_row,
            col=pixel_col,
            broadcast=False,
            offset='auto',
            use=False,
            verbose=False
        )
        
        baseline_storage[chip_name][(pixel_row, pixel_col)] = baseline
        print(f"    Baseline: {baseline:.1f}, Noise width: {noise_width:.1f}")

print(green("Baseline calibration completed"))

print("\n3. Configuring pixels for charge injection...")

# Initialize FIFO and reset system
df = DataFrame()
fifo = FIFO(rb)
time.sleep(1)
fifo.reset()
time.sleep(1)
rb.reset_data_error_count()
rb.enable_etroc_readout()
rb.rerun_bitslip()
fifo.use_etroc_data()

# Reset and configure all chips
for etroc, chip_name, _ in etroc_configs:
    print(f"Resetting {chip_name}...")
    etroc.reset()
    etroc.wr_reg("singlePort", 1)
    
    # Disable all pixels initially
    etroc.wr_reg("disDataReadout", 1, broadcast=True)
    etroc.wr_reg("QInjEn", 0, broadcast=True)
    etroc.wr_reg("enable_TDC", 0, broadcast=True)
    etroc.wr_reg("disTrigPath", 1, broadcast=True)
    etroc.wr_reg("workMode", 0, broadcast=True)
    etroc.wr_reg('triggerGranularity', 1)
    time.sleep(1)

# Configure specific pixels (skip first chip for trigger-only configuration)
for etroc, chip_name, test_pixels in etroc_configs:
    print(f"\nConfiguring {chip_name} pixels...")
    
    for pixel_row, pixel_col in test_pixels:
        print(f"  Configuring pixel ({pixel_row}, {pixel_col})")
        
        # Enable pixel for data readout
        etroc.wr_reg("workMode", 0, row=pixel_row, col=pixel_col, broadcast=False)
        etroc.wr_reg("enable_TDC", 1, row=pixel_row, col=pixel_col, broadcast=False)
        etroc.wr_reg("disDataReadout", 0, row=pixel_row, col=pixel_col, broadcast=False)
        etroc.wr_reg("disTrigPath", 0, row=pixel_row, col=pixel_col, broadcast=False)
        time.sleep(1)
        
        # Set DAC threshold (baseline + margin)
        baseline = baseline_storage[chip_name][(pixel_row, pixel_col)]
        applied_dac = baseline + MANUAL_OFFSET
        etroc.wr_reg('DAC', applied_dac, row=pixel_row, col=pixel_col, broadcast=False)
        
        # Configure charge injection
        etroc.wr_reg("QSel", CHARGE_FC - 1, row=pixel_row, col=pixel_col, broadcast=False)
        etroc.wr_reg("QInjEn", 1, row=pixel_row, col=pixel_col, broadcast=False)
        time.sleep(0.1)
        
        print(f"    DAC: {applied_dac}, Charge: {CHARGE_FC} fC")

print(green("Pixel configuration completed"))

print("\n4. Configuring self-trigger system...")

rb.kcu.write_node(f"READOUT_BOARD_{rb.rb}.TRIG_ENABLE_MASK_0", TRIGGER_ENABLE_MASK)
rb.kcu.write_node(f"READOUT_BOARD_{rb.rb}.TRIG_ENABLE_MASK_1", TRIGGER_DATA_SIZE)
rb.kcu.write_node(f"READOUT_BOARD_{rb.rb}.TRIG_ENABLE_MASK_3", TRIGGER_DELAY_SEL)

print(f"Trigger Enable Mask: 0x{TRIGGER_ENABLE_MASK:X}")
print(f"Trigger Data Size: {TRIGGER_DATA_SIZE}")
print(f"Trigger Delay Sel: {TRIGGER_DELAY_SEL}")

# check elink status
for elink in [0,4,8,12]:
    locked = rb.etroc_locked(elink, slave=False)
    print(f"elink: {elink} locked status: {locked}")

rb.enable_etroc_trigger()
fifo.reset()
time.sleep(1)

print(green("Self-trigger system configured and enabled"))

print(f"\n5. Running data acquisition ({QINJ_COUNT} charge injections)...")

fifo.send_Qinj_only(count=QINJ_COUNT)
time.sleep(2)

try:
    data = fifo.pretty_read(df)
    occupancy = len(data)
    print(green(f"✓ FIFO returned {occupancy} data items"))
    
    if occupancy > 0:
        
        print("\n6. Analyzing data...")
        
        # Initialize data structures
        chip_hits = {}
        chip_data = {}
        
        for etroc, chip_name, test_pixels in etroc_configs:
            chip_hits[chip_name] = {}
            chip_data[chip_name] = {}
            for pixel in test_pixels:
                chip_hits[chip_name][pixel] = 0
                chip_data[chip_name][pixel] = {'toa': [], 'tot': [], 'cal': []}
        
        # Parse data
        header_count = hit_counter = filler_count = trailer_count = 0
        
        for i, event in enumerate(data):
            if event is None or len(event) < 2:
                continue
            
            data_type, event_data = event[0], event[1]
            
            if data_type == 'header':
                header_count += 1
            elif data_type == 'filler':
                filler_count += 1
            elif data_type == 'trailer':
                trailer_count += 1
            elif data_type == 'data':
                hit_counter += 1
                
                # Extract hit information
                toa = event_data.get('toa')
                tot = event_data.get('tot')
                cal = event_data.get('cal')
                row = event_data.get('row_id')
                col = event_data.get('col_id')
                
                # Match to configured pixels
                for etroc, chip_name, test_pixels in etroc_configs:
                    if (row, col) in test_pixels:
                        chip_hits[chip_name][(row, col)] += 1
                        chip_data[chip_name][(row, col)]['toa'].append(toa)
                        chip_data[chip_name][(row, col)]['tot'].append(tot)
                        chip_data[chip_name][(row, col)]['cal'].append(cal)
                        
                        # Show first few hits
                        if chip_hits[chip_name][(row, col)] <= 2:
                            print(green(f"  {chip_name} Pixel({row},{col}) Hit {chip_hits[chip_name][(row, col)]}: "
                                      f"ToA={toa}, ToT={tot}, Cal={cal}"))
        
        # Print summary
        print(f"\nAcquisition Summary:")
        print(f"  Total events: {len(data)}")
        print(f"  Headers: {header_count}")
        print(f"  Hits: {hit_counter}")
        print(f"  Trailers: {trailer_count}")
        print(f"  Fillers: {filler_count}")
        
        # Calculate and display pixel statistics
        print(f"\nPixel Performance Results:")
        test_results = {
            "test_parameters": {
                "timestamp_utc": datetime.now(timezone.utc).isoformat(),
                "charge_fC": CHARGE_FC,
                "qinj_count": QINJ_COUNT,
                "num_ETROCs": len(etroc_configs)
            },
            "chip_results": {},
            "parsed_hits": [event[1] for event in data if event[0] == 'data'],
            "raw_events": data
        }
        
        for etroc, chip_name, test_pixels in etroc_configs:
            for pixel in test_pixels:
                pixel_row, pixel_col = pixel
                hits = chip_hits[chip_name][pixel]
                
                if hits > 0:
                    toa_mean = np.mean(chip_data[chip_name][pixel]['toa'])
                    tot_mean = np.mean(chip_data[chip_name][pixel]['tot'])
                    cal_mean = np.mean(chip_data[chip_name][pixel]['cal'])
                    
                    print(green(f"  {chip_name} Pixel({pixel_row},{pixel_col}): {hits} hits, "
                              f"ToA={toa_mean:.1f}, ToT={tot_mean:.1f}, Cal={cal_mean:.1f}"))
                    
                    # Store results
                    test_results["chip_results"][f"{chip_name}_pixel_{pixel_row}_{pixel_col}"] = {
                        "chip": chip_name,
                        "position": pixel,
                        "hits": hits,
                        "stats": {
                            "toa_mean": toa_mean,
                            "tot_mean": tot_mean,
                            "cal_mean": cal_mean
                        }
                    }
                else:
                    print(red(f"  {chip_name} Pixel({pixel_row},{pixel_col}): No hits detected"))
        
        # ============================================================================
        # 7. SAVE RESULTS
        # ============================================================================
        
        print("\n7. Saving results...")
        output_dir = "Qinj_Output"
        os.makedirs(output_dir, exist_ok=True)
        
        timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        filename = f"cosmic_run_{len(etroc_configs)}_chips_{timestamp}.pkl"
        filepath = os.path.join(output_dir, filename)
        
        try:
            with open(filepath, 'wb') as f:
                pickle.dump(test_results, f)
            print(green(f"Results saved to {filepath}"))
        except Exception as e:
            print(red(f"Failed to save results: {e}"))
            
    else:
        print(red("No data received from FIFO"))
        
except Exception as e:
    print(red(f"Data acquisition failed: {e}"))

# ============================================================================
# 8. CLEANUP SYSTEM
# ============================================================================

print("\n8. Cleaning up system...")

for etroc, chip_name, _ in etroc_configs:
    print(f"Cleaning up {chip_name}...")
    for _ in range(3):
        fifo.reset()
        rb.reset_data_error_count()
        etroc.wr_reg("QInjEn", 0, broadcast=True)
        etroc.wr_reg("disDataReadout", 1, broadcast=True)
        time.sleep(0.1)

print(green("System cleanup completed"))



ETROC COSMIC RUN TEST - QINJ ONLY AND DATA ACQUISITION

1. Generating test pixel configuration...
Test pixel assignments:
  Chip1: [(0, 4)]
  Chip2: [(6, 10), (7, 8)]
  Chip3: [(15, 4), (15, 10)]
  Chip4: [(5, 15)]

2. Calibrating pixel baselines...

Scanning Chip1...
  Calibrating pixel (0, 4)...
    Baseline: 548.0, Noise width: 5.0

Scanning Chip2...
  Calibrating pixel (6, 10)...
    Baseline: 531.0, Noise width: 4.0
  Calibrating pixel (7, 8)...
    Baseline: 527.0, Noise width: 4.0

Scanning Chip3...
  Calibrating pixel (15, 4)...
    Baseline: 548.0, Noise width: 5.0
  Calibrating pixel (15, 10)...
    Baseline: 561.0, Noise width: 5.0

Scanning Chip4...
  Calibrating pixel (5, 15)...
    Baseline: 388.0, Noise width: 3.0
Baseline calibration completed

3. Configuring pixels for charge injection...
Resetting Chip1...
Resetting Chip2...
Resetting Chip3...
Resetting Chip4...

Configuring Chip1 pixels...
  Configuring pixel (0, 4)
    DAC: 598, Charge: 30 fC

Configuring Chip2 pix

In [ ]:
print("ETROC COSMIC RUN DATA ANALYSIS")

# Change this to your data file path
FILEPATH = "Cosmic_Output/cosmic_run_4_chips_2025-07-10_01-54-17.pkl"

# Display options
SHOW_HIT_SAMPLES = 10           # Number of sample hits to show (0 = show all)
SHOW_EVENT_SAMPLES = 10         # Number of sample events to show (0 = show all)
SHOW_OPTIMIZED_SUMMARY = True   # Show additional optimized analysis

with open(FILEPATH, 'rb') as f:
    data = pickle.load(f)

if data is not None:
    print("=== ALL PARAMETERS ===\n")
  
    for main_key in ['test_parameters', 'parsed_hits', 'raw_events', 'chip_results']:
        print(f" {main_key.upper().replace('_', ' ')}:")
        
        if main_key in data:
            if main_key == 'parsed_hits':
                print(f"   Count: {len(data[main_key])}")
                
                # Show sample hits or all hits based on setting
                hits_to_show = data[main_key]
                if SHOW_HIT_SAMPLES > 0:
                    hits_to_show = hits_to_show[:SHOW_HIT_SAMPLES]
                
                for i, hit in enumerate(hits_to_show):
                    print(f"   Hit {i+1}: {hit}")
                
                if SHOW_HIT_SAMPLES > 0 and len(data[main_key]) > SHOW_HIT_SAMPLES:
                    print(f"   ... and {len(data[main_key]) - SHOW_HIT_SAMPLES} more hits")
                    
            elif main_key == 'raw_events':
                print(f"   Count: {len(data[main_key])}")
                
                # Show sample events or all events based on setting
                events_to_show = data[main_key]
                if SHOW_EVENT_SAMPLES > 0:
                    events_to_show = events_to_show[:SHOW_EVENT_SAMPLES]
                
                for i, event in enumerate(events_to_show):
                    print(f"   Event {i+1}: {event}")
                
                if SHOW_EVENT_SAMPLES > 0 and len(data[main_key]) > SHOW_EVENT_SAMPLES:
                    print(f"   ... and {len(data[main_key]) - SHOW_EVENT_SAMPLES} more events")
            
            # Check if data is a dictionary
            elif isinstance(data[main_key], dict):
                for key, value in data[main_key].items():
                    print(f"   {key}: {value}")
            else:
                print(f"   {data[main_key]}")
        else:
            print("   Not found")
        
        print()  # Empty line between sections

    
    if SHOW_OPTIMIZED_SUMMARY:
        
        if 'raw_events' in data:
            events = data['raw_events']
            event_types = {}
            for event in events:
                if event and len(event) >= 1:
                    event_type = event[0]
                    event_types[event_type] = event_types.get(event_type, 0) + 1
            
            print("\nEvent Type Distribution:")
            for event_type, count in event_types.items():
                percentage = (count / len(events)) * 100 if events else 0
                print(f"  {event_type.title()}: {count} ({percentage:.1f}%)")
        
        # Hit analysis by elink
        if 'parsed_hits' in data:
            hits = data['parsed_hits']
            elink_hits = {}
            pixel_hits = {}
            
            for hit in hits:
                elink = hit.get('elink', 'Unknown')
                row = hit.get('row_id', 'N/A')
                col = hit.get('col_id', 'N/A')
                
                elink_hits[elink] = elink_hits.get(elink, 0) + 1
                pixel_key = f"({row},{col})"
                pixel_hits[pixel_key] = pixel_hits.get(pixel_key, 0) + 1
            
            print(f"\nHits by E-link:")
            for elink in sorted(elink_hits.keys()):
                print(f"  Elink {elink}: {elink_hits[elink]} hits")
            
            print(f"\nHits by Pixel:")
            for pixel, count in sorted(pixel_hits.items()):
                print(f"  Pixel {pixel}: {count} hits")
        
        if 'parsed_hits' in data:
            hits = data['parsed_hits']
            toa_values = [hit.get('toa') for hit in hits if hit.get('toa') is not None]
            tot_values = [hit.get('tot') for hit in hits if hit.get('tot') is not None]
            cal_values = [hit.get('cal') for hit in hits if hit.get('cal') is not None]
            
            print(f"\nTiming Statistics:")
            if toa_values:
                print(f"  ToA: min={min(toa_values)}, max={max(toa_values)}, "
                      f"mean={np.mean(toa_values):.1f}, std={np.std(toa_values):.1f}")
            if tot_values:
                print(f"  ToT: min={min(tot_values)}, max={max(tot_values)}, "
                      f"mean={np.mean(tot_values):.1f}, std={np.std(tot_values):.1f}")
            if cal_values:
                print(f"  Cal: min={min(cal_values)}, max={max(cal_values)}, "
                      f"mean={np.mean(cal_values):.1f}, std={np.std(cal_values):.1f}")
        
        if 'test_parameters' in data and 'chip_results' in data:
            params = data['test_parameters']
            total_hits = sum(result.get('hits', 0) for result in data['chip_results'].values())
            injections = params.get('qinj_count', params.get('Qinj_cnt', 1))
            n_pixels = len(data['chip_results'])
            
            expected_hits = injections * n_pixels
            efficiency = (total_hits / expected_hits * 100) if expected_hits > 0 else 0
            
            print(f"\nTest Efficiency:")
            print(f"  Expected hits: {expected_hits} ({injections} inj × {n_pixels} pixels)")
            print(f"  Actual hits: {total_hits}")
            print(f"  Efficiency: {efficiency:.1f}%")

else:
    print("\n❌ Failed to load data. Please check the file path.")




In [ ]:
import pickle
import numpy as np

print("ETROC COSMIC RUN DATA ANALYSIS")

# Change this to your data file path
FILEPATH = "Cosmic_Data_Chunks/session_20250715_115843/chunk_0000.pkl"

# Display options
SHOW_HIT_SAMPLES = 10           # Number of sample hits to show (0 = show all)
SHOW_EVENT_SAMPLES = 10         # Number of sample events to show (0 = show all)
SHOW_OPTIMIZED_SUMMARY = True   # Show additional optimized analysis

with open(FILEPATH, 'rb') as f:
    data = pickle.load(f)

if data is not None:
    print("=== ALL PARAMETERS ===\n")

    for main_key in ['test_parameters', 'statistics', 'parsed_hits', 'raw_events']:
        print(f" {main_key.upper().replace('_', ' ')}:")
        
        if main_key in data:
            if main_key == 'parsed_hits':
                print(f"   Count: {len(data[main_key])}")

                hits_to_show = data[main_key]
                if SHOW_HIT_SAMPLES > 0:
                    hits_to_show = hits_to_show[:SHOW_HIT_SAMPLES]
                
                for i, hit in enumerate(hits_to_show):
                    print(f"   Hit {i+1}: {hit}")
                
                if SHOW_HIT_SAMPLES > 0 and len(data[main_key]) > SHOW_HIT_SAMPLES:
                    print(f"   ... and {len(data[main_key]) - SHOW_HIT_SAMPLES} more hits")
                    
            elif main_key == 'raw_events':
                print(f"   Count: {len(data[main_key])}")

                events_to_show = data[main_key]
                if SHOW_EVENT_SAMPLES > 0:
                    events_to_show = events_to_show[:SHOW_EVENT_SAMPLES]
                
                for i, event in enumerate(events_to_show):
                    print(f"   Event {i+1}: {event}")
                
                if SHOW_EVENT_SAMPLES > 0 and len(data[main_key]) > SHOW_EVENT_SAMPLES:
                    print(f"   ... and {len(data[main_key]) - SHOW_EVENT_SAMPLES} more events")
            
            # Check if data is a dictionary
            elif isinstance(data[main_key], dict):
                for key, value in data[main_key].items():
                    print(f"   {key}: {value}")
            else:
                print(f"   {data[main_key]}")
        else:
            print("   Not found")
        
        print()  # Empty line between sections

    if 'chip_results' in data:
        print(" CHIP RESULTS (LEGACY):")
        chip_results = data['chip_results']
        if isinstance(chip_results, dict):
            for key, value in chip_results.items():
                print(f"   {key}: {value}")
        print()
    
    if SHOW_OPTIMIZED_SUMMARY:
        
        if 'raw_events' in data:
            events = data['raw_events']
            event_types = {}
            for event in events:
                if event and len(event) >= 1:
                    event_type = event[0]
                    event_types[event_type] = event_types.get(event_type, 0) + 1
            
            print("\nEvent Type Distribution:")
            for event_type, count in event_types.items():
                percentage = (count / len(events)) * 100 if events else 0
                print(f"  {event_type.title()}: {count} ({percentage:.1f}%)")
        
        # Hit analysis by elink
        if 'parsed_hits' in data:
            hits = data['parsed_hits']
            elink_hits = {}
            pixel_hits = {}
            
            for hit in hits:
                elink = hit.get('elink', 'Unknown')
                row = hit.get('row_id', 'N/A')
                col = hit.get('col_id', 'N/A')
                
                elink_hits[elink] = elink_hits.get(elink, 0) + 1
                pixel_key = f"({row},{col})"
                pixel_hits[pixel_key] = pixel_hits.get(pixel_key, 0) + 1
            
            print(f"\nHits by E-link:")
            for elink in sorted(elink_hits.keys()):
                print(f"  Elink {elink}: {elink_hits[elink]} hits")
            
            print(f"\nHits by Pixel (showing top 20):")
            sorted_pixels = sorted(pixel_hits.items(), key=lambda x: x[1], reverse=True)
            for i, (pixel, count) in enumerate(sorted_pixels[-20:]):
                print(f"  {i+1}. Pixel {pixel}: {count} hits")
            
            if len(sorted_pixels) > 20:
                print(f"  ... and {len(sorted_pixels) - 20} more pixels with hits")
        
        if 'parsed_hits' in data:
            hits = data['parsed_hits']
            toa_values = [hit.get('toa') for hit in hits if hit.get('toa') is not None]
            tot_values = [hit.get('tot') for hit in hits if hit.get('tot') is not None]
            cal_values = [hit.get('cal') for hit in hits if hit.get('cal') is not None]
            
            print(f"\nTiming Statistics:")
            if toa_values:
                print(f"  ToA: min={min(toa_values)}, max={max(toa_values)}, "
                      f"mean={np.mean(toa_values):.1f}, std={np.std(toa_values):.1f}")
            if tot_values:
                print(f"  ToT: min={min(tot_values)}, max={max(tot_values)}, "
                      f"mean={np.mean(tot_values):.1f}, std={np.std(tot_values):.1f}")
            if cal_values:
                print(f"  Cal: min={min(cal_values)}, max={max(cal_values)}, "
                      f"mean={np.mean(cal_values):.1f}, std={np.std(cal_values):.1f}")
        
        if 'test_parameters' in data:
            params = data['test_parameters']
            if 'parsed_hits' in data:
                total_hits = len(data['parsed_hits'])
                total_time = params.get('total_time_seconds', 0)
                total_pixels = params.get('total_pixels', 0)
                
                print(f"\nCosmic Ray Detection Summary:")
                print(f"  Total running time: {total_time:.1f} seconds")
                print(f"  Total cosmic hits: {total_hits}")
                if total_time > 0:
                    print(f"  Hit rate: {total_hits/total_time:.3f} hits/second")
                if total_pixels > 0:
                    print(f"  Hit rate per pixel: {total_hits/total_pixels:.3f} hits/pixel")
                    if total_time > 0:
                        print(f"  Hit rate per pixel per second: {total_hits/(total_pixels*total_time):.6f} hits/pixel/second")

                if 'failed_pixels_from_baseline' in params:
                    failed_pixels = params['failed_pixels_from_baseline']
                    total_failed = sum(len(failed_list) for failed_list in failed_pixels.values())
                    if total_failed > 0:
                        print(f"\nBaseline Scan Issues:")
                        print(f"  Total pixels with scan failures: {total_failed}")
                        for chip_name, failed_list in failed_pixels.items():
                            if failed_list:
                                print(f"  {chip_name}: {len(failed_list)} failed pixels {failed_list}")
                    else:
                        print(f"\nBaseline Scan: All sample pixels passed")

        if 'statistics' in data:
            stats = data['statistics']
            print(f"\nDetailed Statistics:")
            for key, value in stats.items():
                if key not in ['pixel_hits', 'elink_hits']:
                    print(f"  {key.replace('_', ' ').title()}: {value}")

else:
    print("\n❌ Failed to load data. Please check the file path.")

In [1]:
import pickle

METADATA_FILE = "Cosmic_Data_Chunks/session_20250715_120116/metadata.pkl"

try:
    with open(METADATA_FILE, 'rb') as f:
        metadata = pickle.load(f)
    
    print(f"Successful loaded meta data file: {METADATA_FILE}")
    print(f"data type: {type(metadata)}")
    
    if isinstance(metadata, dict):
        print(f"\n=== contents ===")
        for key, value in metadata.items():
            print(f"  {key}: {value}")
        
except FileNotFoundError:
    print(f"can't find file: {METADATA_FILE}")
except Exception as e:
    print(f"Error: {e}")

print("reading chucnk file now--")
CHUNK_FILE = "Cosmic_Data_Chunks/session_20250724_130428/chunk_0000.pkl"

try:
    with open(CHUNK_FILE, 'rb') as f:
        chunk_data = pickle.load(f)
    
    print(f"Loaded chucnk file: {CHUNK_FILE}")
    print(f"data type: {type(chunk_data)}")
    print(f"Event number: {len(chunk_data) if hasattr(chunk_data, '__len__') else '无法计算'}")
    
    if hasattr(chunk_data, '__len__') and len(chunk_data) > 0:
        print(f"\nfirst 10 events:")
        for i, event in enumerate(chunk_data[:10]):
            print(f"  events {i+1}: {type(event)} - {event}")
    
except FileNotFoundError:
    print(f"could not find file: {CHUNK_FILE}")
except Exception as e:
    print(f"Error: {e}")

Successful loaded meta data file: Cosmic_Data_Chunks/session_20250715_120116/metadata.pkl
data type: <class 'dict'>

=== contents ===
  total_events: 26824496
  total_chunks: 56
  chunk_size: 500000
  session_dir: Cosmic_Data_Chunks/session_20250715_120116
  timestamp: 2025-07-15T12:34:03.970526
reading chucnk file now--
Loaded chucnk file: Cosmic_Data_Chunks/session_20250724_130428/chunk_0000.pkl
data type: <class 'list'>
Event number: 16379

first 10 events:
  events 1: <class 'tuple'> - ('header', {'elink': 0, 'sof': 1, 'eof': 0, 'full': 0, 'any_full': 0, 'global_full': 0, 'l1counter': 49, 'type': 0, 'bcid': 3014, 'raw': '0x3c5c0c4bc6', 'raw_full': '0x1003c5c0c4bc6', 'meta': '0x100'})
  events 2: <class 'tuple'> - ('trailer', {'elink': 0, 'sof': 0, 'eof': 1, 'full': 0, 'any_full': 0, 'global_full': 0, 'chipid': 0, 'status': 0, 'hits': 0, 'crc': 119, 'raw': '0x77', 'raw_full': '0x2000000000077', 'meta': '0x200'})
  events 3: <class 'tuple'> - ('header', {'elink': 4, 'sof': 1, 'eof': 